In [1]:
!jupyter nbconvert --to=python Alk_Glodap_Comparison_surface_CMIP6_style.ipynb

[NbConvertApp] Converting notebook Alk_Glodap_Comparison_surface_CMIP6_style.ipynb to python
[NbConvertApp] Writing 15786 bytes to Alk_Glodap_Comparison_surface_CMIP6_style.py


In [2]:
class Alkcomp_surface:
    '''
    class Alkcomp_surface(runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 GLODAPvar='TAlk',mapproj='pc',savefig=False,layerwise=False)
    '''
    def __init__(self,runname,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 GLODAPvar='TAlk_mmol',
                 mapproj='rob',
                 cmap='viridis',
                 savefig=False,
                 layerwise=False,
                 depth_array=(0,50,200,1000,2000,4000),
                 depth_limit=10,
                 cmap_extension='max',
                 verbose=True,
                 plotting=True,
                 output=False,
                 Taylor=False,
                 get_overview = False):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.GLODAPvar = GLODAPvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit
        self.cmap_extension = cmap_extension
        self.verbose = verbose
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor
        
        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf
        
        from load_interp_GLODAP_python3 import GLODAPdata
        from plot_Taylor_normalized import plt_Taylor_norm
        from plot_Taylor_normalized import plt_Taylor_comp
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)
        
        # load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag= self.resultpath+'/'+self.runname+'.mesh.diag.nc'
        #!ncdump -h $meshdiag

        #diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=self.runname)
        #print(diag)
        #print(diag['Z']) # depth of layers
        #mesh_depths = diag['nz1'].values
        

        # check variables -------------------------------------------------------------------------------
        #NCfesom = self.resultpath + '/Alk.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM'
        unitfesom = 'Alkalinity [mmol m$^{-3}$]' 

        # load FESOM data -------------------------------------------------------------------------------------
        Alkfesom = pf.get_data(resultpath, "Alk", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)

        # load raw Glodap data -------------------------------------------------------------------------------------
        
        unitglodap = 'Alkalinity [mmol m$^{-3}$]'
        labelglodap = 'GLODAP'

        
        # interpolate Glodap data -------------------------------------------------------------------------------------
        GLODAP_input = GLODAPdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.GLODAPvar, get_overview=False)
        glodap_int = GLODAP_input.glodap_int 
        

        # apply sea mask to Glodap as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        glodap_int_ma = np.copy(glodap_int)
        glodap_int_ma[Alkfesom == 0] = 0

        # plot Glodap and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if plotting:
                    fig = plt.figure(figsize=(14,7), constrained_layout=True)
                    axes = fig.subplot_mosaic(
                                """
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                                subplot_kw=dict(projection=self.mapproj))

#                     m1 = axes['A']
#                     levels = np.arange(1800,2520,20)
#                     f1 = pf.subplot(mesh, m1, [Alkfesom[:,i]],
#                                 levels = levels,
#                                 units=unitglodap, 
#                                 mapproj=self.mapproj, # robinson projection takes more time!
#                                 cmap = self.cmap,
#                                 cmap_extension=self.cmap_extension,
#                                 titles=labelfesom+'\n ({0} m)'.format(depth_limit),
#                                 box=box,
#                                )

#                     m2 = axes['B']
#                     f2 = pf.subplot(mesh, m2, [glodap_int_ma[:,i]], 
#                                 levels = levels,
#                                 units=unitglodap, 
#                                 mapproj=self.mapproj, # robinson projection takes more time!
#                                 cmap = self.cmap,
#                                 cmap_extension=self.cmap_extension,
#                                 titles=labelwoa+'\n ({0} m)'.format(depth_limit),
#                                 box=box,
#                                )

#                     cbar1_ax = fig.add_axes([0.13, 0.55, 0.76, 0.02])
#                     cbar1 = fig.colorbar(f1,
#                                     cax = cbar1_ax, 
#                                     orientation = 'horizontal',
#                                     fraction=0.046, pad=0.04) 
#                     cbar1.set_label(unitfesom, fontsize=18)
#                     cbar1.ax.tick_params(labelsize=18)

                    m3 = axes['C']

                    levels_diff = np.arange(-200,210,10)
                    f3 = pf.subplot(mesh, m3, [Alkfesom[:,i]-glodap_int_ma[:,i]], 
                                rowscol= (1,1),
                                levels = levels_diff,
                                units=unitglodap, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'bwr',
                                cmap_extension='both',
                                titles='FESOM2.1-REcoM3 - GLODAP\n ({0} m)'.format(depth_limit),
                                box=box,
                               )

                    fig.subplots_adjust(bottom=0.02)
                    cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                    cbar2 = fig.colorbar(f3,
                                    cax = cbar2_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar2.set_label(unitfesom, fontsize=18)
                    cbar2.ax.tick_params(labelsize=18)
                    
                    
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'Alk_surface_GLODAP_CH'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}m.png'.format(depth_limit), 
                                dpi = 300, bbox_inches='tight')
                    plt.show(block=False)
  
                if Taylor:
                    # statistics  -------------------------------------------------------------------------------------
                    # preparation of datasets
                    if np.isnan(np.min(glodap_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
                    if np.isnan(np.min(DICfesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                    title = 'Taylor Diagram for Alkalinity'
                    plt_Taylor_comp(glodap_int_ma,Alkfesom,mask=True,title=title, depth_array=depth_array, mesh=mesh)
                    
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'Alk_surface_GLODAP_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_layerwise.png', 
                                        dpi = 300, bbox_inches='tight')
                    plt.show(block=False)
                    
                if output:
                    print('Only return non-layerwise output')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            # get level depth index closest to depth_limit
            i_depth_limit = pf.ind_for_depth(self.depth_limit, mesh)
            
            # corresponding level depth
            depth_limit_level = - mesh.zlev[i_depth_limit]
            
            # corresponding layer depth (where concentrations are defined):
            depth_limit_layer = - GLODAP_input.layer_depths[i_depth_limit]
            
            # print overview of chosen depth
            if(self.verbose):
                print('Alkalinity as mean over depth \nwith max layer depth = {0} \n(level depth = {1}m, mesh index {2})'.format(
                    depth_limit_layer,depth_limit_level,i_depth_limit))
            
            Alkfesom_mean = np.mean(Alkfesom[:,:i_depth_limit], axis = 1)
            glodap_int_ma_mean = np.mean(glodap_int_ma[:,:i_depth_limit], axis = 1)
            
            if(self.verbose):
                print('\nPlotting Alkalinity as mean over depth\nFESOM min = {0}, max = {1}\nGlodap min = {2}, max = {3}'.format(
                        np.nanmin(Alkfesom_mean),np.nanmax(Alkfesom_mean),
                        np.nanmin(glodap_int_ma_mean),np.nanmax(glodap_int_ma_mean)))
    
            if plotting:
                fig = plt.figure(figsize=(14,8), constrained_layout=True)
                axes = fig.subplot_mosaic(
                            """
                            CC
                            """,
                            gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                            subplot_kw=dict(projection=self.mapproj))
                    
#                 m1 = axes['A']
#                 levels = np.arange(1800,2520,20)
#                 f1 = pf.subplot(mesh, m1, [Alkfesom_mean],
#                             levels = levels,
#                             units=unitglodap, 
#                             mapproj=self.mapproj, # robinson projection takes more time!
#                             cmap = self.cmap,
#                             cmap_extension=self.cmap_extension,
#                             titles=labelfesom+'\n (0-{0} m)'.format(depth_limit),
#                             box=box,
#                            )
                    
#                 m2 = axes['B']
#                 f2 = pf.subplot(mesh, m2, [glodap_int_ma_mean], 
#                             levels = levels,
#                             units=unitglodap, 
#                             mapproj=self.mapproj, # robinson projection takes more time!
#                             cmap = self.cmap,
#                             cmap_extension=self.cmap_extension,
#                             titles=labelglodap+'\n (0-{0} m)'.format(depth_limit),
#                             box=box,
#                            )
                    
#                 cbar1_ax = fig.add_axes([0.13, 0.55, 0.76, 0.02])
#                 cbar1 = fig.colorbar(f1,
#                                 cax = cbar1_ax, 
#                                 orientation = 'horizontal',
#                                 fraction=0.046, pad=0.04)
#                 cbar1.set_label(unitfesom, fontsize=18)
#                 cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-200,210,10)
                f3 = pf.subplot(mesh, m3, [Alkfesom_mean-glodap_int_ma_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitglodap, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'bwr',
                            cmap_extension='both',
                            titles='FESOM2.1-REcoM3 - GLODAP'+'\n (0-{0} m)'.format(depth_limit),
                            box=box,
                           )
                
                fig.subplots_adjust(bottom=0.02)
                cbar2_ax = fig.add_axes([0.13, 0.03, 0.76, 0.03])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.01) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)

                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'Alk_surface_GLODAP_CH'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False)
                
                
            if Taylor:
                # statistics  -------------------------------------------------------------------------------------            
                # preparation of datasets
                if np.isnan(np.min(glodap_int_ma_mean)): print('WARNING: The interpolated Glodap field contains NaNs')
                if np.isnan(np.min(Alkfesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs')

                title = 'Taylor Diagram for Alk \n(mean over depth, max = {0}m)'.format(depth_limit_layer)
                plt_Taylor_norm(glodap_int_ma_mean,Alkfesom_mean,mask=True,title=title)
                
                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'Alk_surface_Glodap_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False)  
                
            if output:
                    self.alk_fesom = Alkfesom_mean
                    self.alk_phc   = glodap_int_ma_mean